In [1]:
import torch
from cgmodel import CycleGan
from src.UnpairedDataset import UnpairedDataset
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np

In [2]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(286, Image.BICUBIC),
        transforms.RandomCrop(256),
        transforms.ToTensor(),
        transforms.Normalize([.5, .5, .5], [.5, .5, .5])
    ]),
    'test': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([.5, .5, .5], [.5, .5, .5])
    ]),
}
root = 'C:/Users/Deepak H R/Desktop/data/monet2photo/'
test = UnpairedDataset(root, 'test', transforms=data_transforms['test'])
test = DataLoader(test, batch_size=4)
model = CycleGan.load_from_checkpoint('epoch_kaggle4.ckpt')

Found 121 images of testA and 751 images of testB
Initialized network with normal


In [3]:
batch = next(iter(test))
imgA, imgB = batch['A'], batch['B']
with torch.no_grad():
    fakeB = model.genX(imgA)
    fakeA = model.genY(imgB)

In [4]:
def toImage(x):
    # converts tensor in range [-1,1] to a pil image
    x = x.numpy()
    x = np.transpose(x, (1,2,0))
    x = (x + 1)*127.5
    x = x.astype('uint8')
    return Image.fromarray(x)

[imgA, imgB, fakeA, fakeB] = [[toImage(f) for f in X] for X in [imgA, imgB, fakeA, fakeB]]

for i, (r1, r2, f1, f2) in enumerate(zip(imgA, imgB, fakeA, fakeB)):
    r1.save(f'images/reala_{i}.jpg')
    r2.save(f'images/realb_{i}.jpg')
    f1.save(f'images/fakea_{i}.jpg')
    f2.save(f'images/fakeb_{i}.jpg')  